# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [14]:
weather_city = pd.read_csv("city_weather.csv")
weather_city_df = pd.DataFrame(weather_city)
weather_city_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lebu,-37.62,-73.65,44.71,69,21,8.32,CL,1595545488
1,Punta Arenas,-53.15,-70.92,39.09,80,75,13.87,CL,1595545317
2,Hithadhoo,-0.60,73.08,82.96,69,85,7.63,MV,1595545488
3,Pevek,69.70,170.31,47.16,73,19,11.59,RU,1595545488
4,Mataura,-46.19,168.86,39.00,83,80,3.00,NZ,1595545488


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 # Store latitude and longitude in locations
locations = weather_city_df[["Latitude", "Longitude"]]

# Convert humidity to float
humidity = weather_city_df["Humidity"].astype(float)

In [4]:
 # Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=16,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather_df = weather_city_df.loc[(weather_city_df["Temperature"] < 80) & (weather_city_df["Temperature"] > 70)\
                                      & (weather_city_df["Wind Speed"] < 10) & (weather_city_df["Cloudiness"] == 0)]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
5,Kiryat Gat,31.61,34.76,73.71,63,0,2.64,IL,1595545488
82,Melilla,35.29,-2.94,76.84,69,0,2.24,ES,1595545501
88,Guiratinga,-16.35,-53.76,73.92,32,0,3.87,BR,1595545503
103,Mahon,39.89,4.27,73.40,78,0,3.36,ES,1595545505
166,Bayir,37.27,28.22,71.60,78,0,2.24,TR,1595545516
192,Canals,38.96,-0.58,77.25,60,0,1.68,ES,1595545521
197,Vila Velha,-20.33,-40.29,70.95,83,0,4.83,BR,1595545305
206,Bāfq,31.61,55.41,77.99,16,0,1.81,IR,1595545523
245,Naples,40.83,14.25,73.60,69,0,3.36,IT,1595545260
251,Jinzhou,41.11,121.14,79.47,50,0,4.61,CN,1595545531


In [6]:
# Hotel Map

# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = ideal_weather_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
5,Kiryat Gat,IL,31.61,34.76,
82,Melilla,ES,35.29,-2.94,
88,Guiratinga,BR,-16.35,-53.76,
103,Mahon,ES,39.89,4.27,
166,Bayir,TR,37.27,28.22,
192,Canals,ES,38.96,-0.58,
197,Vila Velha,BR,-20.33,-40.29,
206,Bāfq,IR,31.61,55.41,
245,Naples,IT,40.83,14.25,
251,Jinzhou,CN,41.11,121.14,


In [13]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
5,Kiryat Gat,IL,31.61,34.76,רגע בנחת
82,Melilla,ES,35.29,-2.94,Hôtel Anfora
88,Guiratinga,BR,-16.35,-53.76,Hotel Angra do Cerrado - Guiratinga MT
103,Mahon,ES,39.89,4.27,ARTIEM Capri
166,Bayir,TR,37.27,28.22,Buban Otel
192,Canals,ES,38.96,-0.58,Hostal Casa La Vuelta
197,Vila Velha,BR,-20.33,-40.29,Hotel Vitória Palace
206,Bāfq,IR,31.61,55.41,Alamdar Hotel
245,Naples,IT,40.83,14.25,Renaissance Naples Hotel Mediterraneo
251,Jinzhou,CN,41.11,121.14,Sheraton Jinzhou Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(hotel_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))